# Automatic Sandpit Refinement for D-Flow FM

This notebook provides an automated workflow for refining unstructured grids around sandpit areas in D-Flow FM models. The refinement process uses Casulli refinement to gradually transition from coarse background resolution to fine target resolution.

## Workflow Overview

1. **Configuration**: Set file paths and refinement parameters
2. **Load Grid and Polygons**: Load the grid and create/load sandpit polygons
3. **Plan Refinement**: Analyze grid resolution and generate refinement zones
4. **Execute Refinement**: Apply Casulli refinement to the grid
5. **Monitor Quality** (Optional): Analyze grid quality metrics
6. **Setup Partitioned Model**: Create partitioned model with refined grid
7. **Modify Restart Variables**: Apply sandpit excavation to bed levels in restart files

## Requirements

- `dfm_tools`
- `meshkernel`
- `numpy`
- `matplotlib`
- `shapely`

In [ ]:
# Step 0: Configuration and Setup
import sys
import os

# Add project root to Python path (for local installations)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Print location of the environment / interpreter
print(f"Python interpreter: {sys.executable}")
    
import numpy as np
import dfm_tools as dfmt
import matplotlib.pyplot as plt
import xugrid as xu
import xarray as xr
import psutil
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial.distance import cdist
import scipy.interpolate
import meshkernel
import re
from pathlib import Path

# Import utility functions
from src.polygon_utils import InteractivePolygonDrawer, load_pol_file, save_pol_file, generate_refinement_polygons, expand_polygon_outward, add_shape_to_polygon
from src.refinement_utils import compute_refinement_steps, apply_casulli_refinement, print_refinement_summary
from src.visualization_utils import plot_grid, is_codespace, plot_restart_results
from src.monitoring_utils import analyze_grid_quality, plot_grid_quality
from src.restart_utils import setup_partitioned_model, load_restart_files, save_restart_files
from src.netcdf_utils import export_refined_grid

# INPUT: Filenames
nc_file ='dcsm_0_125nm_2ref_bathygr7_RGFGRID_net.nc' #'DCSM-FM_0_5nm_grid_20220310_depth_20220517_net.nc'

# INPUT: Toggle options
visualization_bool = False          # Set to True to enable visualization
monitor_quality = False             # Set to True if you to analyze the quality of the grid
use_existing_pol_file = True        # Set to True to use an existing polygon file (no new polygons will be created)
plot_bathymetry = False             # Set to True to plot bathymetry in the background of interactive plot
setup_partitioned_model_bool = True # Set to True to setup partitioned model

# INPUT: Refinement parameters
target_resolution = 100  # Target resolution in meters for the finest refinement level
buffer_around_sandpit = 250  # Buffer around sandpit polygons in meters
N = 6  # Number of transition cells

# INPUT: DFM model and installation paths
model_dir = Path("../data/model/coldstart_test")
original_mdu = "DCSM-FM_0_5nm.mdu"
dimr_config = "dimr_config.xml"
dflowfm_exe = Path("p:/11211460-msc-gw-coupling/02_engines/2025.01/x64/bin/run_dflowfm.bat")
dimr_parallel_exe = Path("p:/11211460-msc-gw-coupling/02_engines/2025.01/x64/bin/run_dimr_parallel.bat")
n_partitions = 10

# INPUT: Digging parameters
dig_depth = 5.0        # meters to lower bed level
slope = 0.03            # slope ratio for smooth transitions (default: 0.1)

# Auto-detect correct data path based on current working directory
if os.path.basename(os.getcwd()) == 'notebooks':
   # Running from notebooks directory (Codespace)
   data_path = os.path.join('..', 'data')
else:
   # Running from project root (local Jupyter)
   data_path = 'data'

# INPUT: File paths (input & output)
nc_path = os.path.join(data_path, 'input', nc_file)
input_pol_file = os.path.join(data_path, 'input', 'sandpits.pol')
output_pol_file = os.path.join(data_path, 'output', 'sandpits.pol')
xyz_output_path = os.path.join(data_path, 'output', 'bathymetry.xyz')
output_dir = os.path.join(data_path, 'output')
os.makedirs(output_dir, exist_ok=True)

# Environment detection
if is_codespace():
    print("Running in GitHub Codespace")
else:
    print("Running locally")

print("Configuration loaded")
print(f"   Target: {target_resolution}m | Buffer: {buffer_around_sandpit}m | Transitions: {N}")
print(f"   Model: {model_dir} | Partitions: {n_partitions}")
print(f"   Digging: {dig_depth}m depth | {slope} slope ratio")

## Step 1: Load Grid and Define Sandpit Polygons

This step loads the D-Flow FM grid and either:
- Loads existing sandpit polygons from a .pol file
- Opens an interactive drawing tool to create new polygons

The polygons define the areas where refinement will be applied.

**Note**: Interactive polygon drawing is only available when running locally, not in Codespace.

In [ ]:
# Step 1: Load Grid and Create/Load Polygons

# Check if NetCDF file exists
if not os.path.exists(nc_path):
    raise FileNotFoundError(f"Required file not found: {nc_path}")

# Load grid data
print("Loading grid...")
ugrid = dfmt.open_partitioned_dataset(nc_path)
ugrid_original = dfmt.open_partitioned_dataset(nc_path)  # Keep original for later comparison

# Convert to meshkernel objects for consistent plotting and refinement
mk_object = ugrid.grid.meshkernel
mk_backup = ugrid_original.grid.meshkernel  # Keep backup

# Handle polygon input/creation
if use_existing_pol_file:
    # Load existing polygon file
    if os.path.exists(input_pol_file):
        polygons = load_pol_file(input_pol_file)
        print(f"Loaded {len(polygons)} sandpit polygons from file")
    else:
        print(f"File {input_pol_file} not found, switching to interactive mode")
        use_existing_pol_file = False

if not use_existing_pol_file:
    # Interactive polygon creation
    if is_codespace():
        print("Interactive polygon drawing not available in Codespace")
        print("   Please create a sandpits.pol file or run locally for interactive drawing")
        raise RuntimeError("Interactive mode not supported in Codespace")
    
    # Create an interactive plot
    %matplotlib tk
    print("Opening interactive polygon drawing tool...")
    print("   Instructions: RIGHT CLICK â†' add vertex | ENTER â†' finish polygon | Close window when done")
    
    drawer = InteractivePolygonDrawer(ugrid, nc_path, plot_bathymetry)
    polygons = drawer.draw_polygons()
    
    if polygons:
        output_file = os.path.join(output_dir, 'sandpits.pol')
        save_pol_file(polygons, output_file)
        print(f"Saved {len(polygons)} polygons to {output_file}")


# Uncomment to add one new shape to the existing polygon file
add_shape_to_polygon(input_pol_file, output_pol_file, 'rectangle', 4.0, 52.2, width=0.03, height=0.02, rotation=45)
polygons = load_pol_file(output_pol_file)  # Reload polygons after adding new shape

## Step 2: Plan Refinement Strategy

This step:
1. Analyzes the current grid resolution within the sandpit polygons
2. Calculates the number of refinement steps needed
3. Generates refinement zones with automatic overlap detection and merging
4. Visualizes the refinement plan

The refinement zones are created from coarse (outer) to fine (inner) resolution.

In [ ]:
# Step 2: Plan Refinement Strategy

# Analyze grid resolution and compute refinement steps
print("Analyzing grid resolution...")
refinement_params = compute_refinement_steps(ugrid, target_resolution, polygons)

# Generate refinement polygons with overlap merging
print("Generating refinement zones...")
(all_refinement_polygons, all_original_polygons, 
 buffer_polygons, expansions) = generate_refinement_polygons(
    polygons, refinement_params, buffer_around_sandpit, N)

# Store original buffer polygons for visualization
original_buffer_polygons = []
for i, polygon in enumerate(polygons):
    polygon_array = np.array(polygon)
    center_lat = np.mean(polygon_array[:, 1])
    expanded_polygon = expand_polygon_outward(polygon, buffer_around_sandpit, center_lat)
    original_buffer_polygons.append(expanded_polygon)

# Visualize refinement plan
if visualization_bool:
    print("Visualizing refinement plan...")
    if not is_codespace():
        %matplotlib inline
    plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
            buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
            original_buffer_polygons=original_buffer_polygons,
            title='Refinement Plan: Sandpit Polygons and Merged Refinement Zones')
    plt.show()

## Step 3: Execute Grid Refinement

This step applies Casulli refinement to the meshkernel object using the generated refinement zones. The refinement is applied from outside to inside (coarse to fine) to ensure smooth transitions.

After refinement, the refined grid is visualized showing the final mesh structure.

In [ ]:
# Step 3: Execute Grid Refinement

# Apply Casulli refinement
print("Executing refinement...")
apply_casulli_refinement(mk_object, all_refinement_polygons)

# Visualize refined grid
if visualization_bool:
    print("Visualizing refined grid...")
    if not is_codespace():
        %matplotlib inline
    plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
            buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
            title='Refined Grid: Final Result with Casulli Refinement')

# Print refinement summary
print_refinement_summary(polygons, all_refinement_polygons, 
                        refinement_params['envelope_sizes_m'], 
                        refinement_params['n_steps'], buffer_polygons)

## Step 4: Monitor Grid Quality (Optional)

This optional step analyzes the quality of the refined grid by examining:
- **Resolution**: Face areas converted to characteristic lengths
- **Smoothness**: Ratio of adjacent cell sizes (target < 1.4)
- **Orthogonality**: Deviation from 90° angles (target < 0.01)

**Note**: This analysis can be computationally intensive for large grids, especially in Codespace.

In [ ]:
# Step 4: (Optional) Monitor Grid Quality
if monitor_quality:
    print("Analyzing grid quality metrics...")
    quality_data = analyze_grid_quality(mk_object, ugrid_original, all_refinement_polygons, polygons)
    
    print("Creating quality visualization...")
    if not is_codespace():
        %matplotlib inline
        plot_grid_quality(quality_data, all_refinement_polygons, target_resolution)
    else:
        print("Apparently the kernel crashes when trying to plot this in a Codespace environment...")

## Step 5: Export Refined Grid

This step exports the refined grid to a RGFGRID-compatible NetCDF file with automatic versioning to prevent overwriting existing files. The saved file contains complete UGRID topology including all connectivity arrays required by D-Flow FM.

This refined grid serves as the foundation for the partitioned model setup and subsequent restart file generation.

In [ ]:
# Step 5: Export refined grid to NetCDF with proper naming and compatibility

output_refined_nc, ugrid_complete = export_refined_grid(
    mk_object=mk_object,
    original_nc_file=nc_file, 
    output_dir=output_dir,
    suffix="_ref"
)

## Step 6: Setup Partitioned Model

This step creates a partitioned D-Flow FM model using the refined grid:

1. **Create temporary files**: Generate temporary MDU and DIMR config files with appropriate settings
2. **Partition grid**: Apply domain decomposition using D-Flow FM partitioning
3. **Generate restart files**: Run a short model simulation to create initial restart files

The process creates all necessary files for running the refined model in parallel.

In [ ]:
# Step 6: Setup Partitioned Model
if setup_partitioned_model_bool:
    print(f"Setting up partitioned model with {n_partitions} domains...")
    
    # Run complete partitioning workflow
    results = setup_partitioned_model(
        model_dir=model_dir,
        original_mdu=model_dir / original_mdu,
        dimr_config=model_dir / dimr_config,
        refined_grid_nc=Path(output_refined_nc),
        dflowfm_exe=dflowfm_exe,
        dimr_parallel_exe=dimr_parallel_exe,
        n_partitions=n_partitions
    )

else:
    print("Partitioned model setup skipped (set setup_partitioned_model_bool=True to enable)")

## Step 7: Modify Variables in Restart Files

This final step applies sandpit excavation by modifying the bed levels in the restart files generated from Step 6:

1. **Load restart files**: Read all partition restart files from the temporary model run
2. **Modify bed levels**: Apply excavation within sandpit polygons using configured dig depth and slope parameters
3. **Save modified files**: Export the modified restart files for use in production runs
4. **Visualize results**: Plot the bed level modifications to verify the excavation pattern

The modified restart files can be used as initial conditions for D-Flow FM simulations with the excavated sandpit geometry.

In [ ]:
# Step 7: Modify Variables in Restart Files

# 1. Load restart files
temp_mdu_name = results['temp_mdu'].stem  # DCSM-FM_0_5nm_temp
restart_folder = model_dir / f"DFM_OUTPUT_{temp_mdu_name}"
datasets = load_restart_files(restart_folder)

# 2. Modify variables
from src.modify_bedlevel import modify_bed_levels
modified_datasets = modify_bed_levels(datasets, polygons, dig_depth, slope)

# Add other variable modifications here:
# from src.modify_variable_template import modify_water_level
# modified_datasets = modify_water_level(modified_datasets, polygons, level_change_m=0.1)

# 3. Save modified restart files
restart_output_dir = Path(output_dir) / "restart"
saved_files = save_restart_files(modified_datasets, restart_output_dir)

# 4. Plot results
plot_restart_results(modified_datasets, polygons, "Modified Restart Files")

print(f"Saved {len(saved_files)} modified restart files to {restart_output_dir}")